In [1]:
import string #for strings  
import re #regular expressions  
import pandas as pd #for loading data,reading and manipulating  
import matplotlib.pyplot as plt #for plotting  
#sklearn is used to build machine learning machine  
from sklearn import feature_extraction  
from sklearn import linear_model  
from sklearn.naive_bayes import MultinomialNB  
from sklearn.linear_model import LogisticRegression  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier  
from sklearn import pipeline  
from sklearn.model_selection import train_test_split  
from sklearn import metrics  
import itertools 


# Loading datasets

In [10]:
#loading datasets
eng_df = pd.read_csv("C:\ML Projects\code and datsets\europarl_raw\europarl_raw\english\ep-00-01-17.en", "utf-8",engine='python', header=None, names=["English"])    
ger_df= pd.read_csv("C:\ML Projects\code and datsets\europarl_raw\europarl_raw\german\ep-00-01-17.de","utf-8", engine='python',header=None,names=["German"])    
#fre_df= pd.read_csv("C:\ML Projects\code and datsets\europarl_raw\europarl_raw\french\ep-00-01-20.fr","utf-8",engine='python',header=None,names=["French"])


# Removing special characters

In [11]:
print ("THE SPECIAL CHARECTERS THAT ARE REMOVED ARE:",end="\n")    
for char in string.punctuation:  
    print(char,end=" ")  
translate_table=dict((ord(char),None) for char in string.punctuation)  


THE SPECIAL CHARECTERS THAT ARE REMOVED ARE:
! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

### data preprocessing for english raw data

In [15]:
data_eng=[]  
lang_eng=[]  
for i,line in eng_df.iterrows():  
    line=line['English']  
    if len(line) !=0:  
        line = line.lower()  
        line = re.sub(r"\d+","",line)  
        line=line.translate(translate_table)  
        data_eng.append(line)  
        lang_eng.append("english")  

In [16]:
data_ger=[]  
lang_ger=[]  
for i,line in ger_df.iterrows():  
    line=line['German']  
    if len(line) !=0:  
        line = line.lower()  
        line = re.sub(r"\d+","",line)  
        line=line.translate(translate_table)  
        data_eng.append(line)  
        lang_eng.append("german")  

In [17]:
df=pd.DataFrame({"Text":data_eng+data_ger, "language":lang_eng+lang_ger})  
print(df.shape)  

(1829, 2)


### SPLITTING DATA INTO TRAIN AND TEST SETS(80:20)

In [18]:
print ("ALLOCATION OF TRAIN AND DATA SETS",end="\n")  
x,y = df.iloc[:,0],df.iloc[:,1]  
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)    
print(x_train.shape)  
print(x_test.shape)  
print(y_train.shape)  
print(y_test.shape)  

ALLOCATION OF TRAIN AND DATA SETS
(1463,)
(366,)
(1463,)
(366,)


## VECTORIZATION AND MODEL FITTING PIPELINE  

In [19]:
vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),analyzer='char')    
pipe_lr_r13 = pipeline.Pipeline([ ('vectorizer',vectorizer), ('clf',linear_model.LogisticRegression()) ])  
pipe_lr_r13.fit(x_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

## MODEL PREDECTION

In [20]:
y_predicted = pipe_lr_r13.predict(x_test)

## MODEL EVOLUTION 

In [21]:
print("ACCURACY SCORE",end="\n")  
acc = (metrics.accuracy_score(y_test,y_predicted))*100  
print(acc,'%')  
matrix = metrics.confusion_matrix(y_test,y_predicted)  
print('confusion matrix:\n',matrix) 

ACCURACY SCORE
100.0 %
confusion matrix:
 [[191   0]
 [  0 175]]


In [25]:
import pickle
file=open('LRModel.pckl','wb')
pickle.dump(pipe_lr_r13,file)

In [26]:
def lang_detect(text):  
    import numpy as np  
    import string  
    import re  
    import pickle  
    translate_table = dict((ord(char), None) for char in string.punctuation)    
    global lrLangDetectModel  
    lrLangDetectFile = open('LRModel.pckl','rb')  
    lrLangDetectModel= pickle.load(lrLangDetectFile)  
    lrLangDetectFile.close()  
    text=" ".join(text.split())  
    text=text.lower()  
    text=re.sub(r"\d+","",text)  
    text=text.translate(translate_table)  
    pred=lrLangDetectModel.predict([text])  
    prob=lrLangDetectModel.predict_proba([text])  
    return pred[0]  
print(lang_detect("Hello i just built my own language detection model"))  
print(lang_detect("hello ,ich habe gerade main eigines sprancherkennugsmodel erstellt"))  
#print(lang_detect(" Bonjourm je viens de crier mon propre"))


english
german
